In [57]:
import os

In [58]:
%pwd

'/Users/andrewbelyaev/UIUC/MLOps-project'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/andrewbelyaev/UIUC/MLOps-project'

In [59]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/siiyayu/MLOps-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="=siiyayu"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4f9dd36a95bb3913099f776d1307229883c62805"

In [60]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [61]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [62]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/siiyayu/MLOps-project.mlflow",
        )

        return model_evaluation_config


In [69]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [70]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        dagshub.init(repo_owner='siiyayu', repo_name='MLOps-project', mlflow=True)


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [71]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-04 18:00:33,561: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-04 18:00:33,564: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-04 18:00:33,565: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-04 18:00:33,566: INFO: common: created directory at: artifacts]
[2025-01-04 18:00:33,567: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-04 18:00:33,787: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/siiyayu/MLOps-project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "siiyayu/MLOps-project"

[2025-01-04 18:00:33,806: INFO: helpers: Initialized MLflow to track repo "siiyayu/MLOps-project"]


Repository siiyayu/MLOps-project initialized!

[2025-01-04 18:00:33,807: INFO: helpers: Repository siiyayu/MLOps-project initialized!]
[2025-01-04 18:00:33,981: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/01/04 18:00:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/tj/3h1b3qks39x9tkrzl6ptzs340000gn/T/tmpscv9e192/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.6.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/Users/andrewbelyaev/UIUC/MLOps-project/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2025/01/04 18:00:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
